## Using the MRJob Class below  calculate the  KL divergence of the following two objects.

In [1]:
%%writefile kltext.txt
1.Data Science is an interdisciplinary field about processes and systems to extract knowledge or insights from large volumes of data in various forms (data in various forms, data in various forms, data in various forms), either structured or unstructured,[1][2] which is a continuation of some of the data analysis fields such as statistics, data mining and predictive analytics, as well as Knowledge Discovery in Databases.
2.Machine learning is a subfield of computer science[1] that evolved from the study of pattern recognition and computational learning theory in artificial intelligence.[1] Machine learning explores the study and construction of algorithms that can learn from and make predictions on data.[2] Such algorithms operate by building a model from example inputs in order to make data-driven predictions or decisions,[3]:2 rather than following strictly static program instructions.

Overwriting kltext.txt


## MRjob class for calculating pairwise similarity using K-L Divergence as the similarity measure

Job 1: create inverted index (assume just two objects) <P>
Job 2: calculate the similarity of each pair of objects 

In [2]:
import numpy as np
np.log(3)

1.0986122886681098

In [5]:
%%writefile kldivergence.py
from mrjob.job import MRJob
import re
import numpy as np
class kldivergence(MRJob):
    def mapper1(self, _, line):
        index = int(line.split('.',1)[0])
        letter_list = re.sub(r"[^A-Za-z]+", '', line).lower()
        count = {}
        for l in letter_list:
            if count.has_key(l):
                count[l] += 1
            else:
                count[l] = 1
        for key in count:
            yield key, [index, count[key]*1.0/len(letter_list)]

    def reducer1(self, key, values):
        p_prob = 0.0
        q_prob = 0.0
        probabilities = []
        current_key = None
        for value in values:
            doc = value[0]
            cond = float(value[1])
            if current_key:
                if current_key != key:
                    probabilities.append(p_prob*np.log((p_prob+1)/(q_prob+24)))
                    current_key = key
                    if doc == '1':
                        p_prob = cond
                    if doc == '2':
                        q_prob = cond                    
                else:
                    if doc == '1':
                        p_prob = cond
                    if doc == '2':
                        q_prob = cond
            else:
                current_key = key
                if doc == '1':
                    p_prob = cond
                if doc == '2':
                    q_prob = cond
        probabilities.append(p_prob*np.log((p_prob+1)/(q_prob+24)))
        yield None, probabilities
        
    def reducer2(self, key, values):
        kl_sum = 0.0
        for value in values:
            kl_sum = kl_sum + float(value)
        yield None, kl_sum
            
    def steps(self):
        return [self.mr(mapper=self.mapper1,
                        reducer=self.reducer1),
                 self.mr(reducer=self.reducer2)]

if __name__ == '__main__':
    kldivergence.run()

Overwriting kldivergence.py


In [6]:
%reload_ext autoreload
%autoreload 2
from kldivergence import kldivergence
mr_job = kldivergence(args=['kltext.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

TypeError: float() argument must be a string or a number